# Imports

In [95]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np
import scipy.stats as stats
import random
import scipy.stats as stats

# Primordial filter
Filters all publication according to the manual categorizatino of the publication into relevant or irrelevant based on the eligibility criteria

In [96]:
extraction=pd.read_csv('./input/extraction_cleaned.csv')

extractioneligible=pd.read_csv('./input/extraction_cleaned_eligible_oa.csv')

In [97]:
test=list(extractioneligible.PMID)
list_string = map(str, test) 

searchstring=' OR '.join(list_string)
with open('./output/relevant_search_pmids.txt', 'w') as f:
    f.write(searchstring)

#### then you run the search to extract the pmcids of the papers. 
To filter out the ones that are not indexed in PMC subset you have to search using the relevant_search_pmids.txt and add the term `(pubmed pmc open access[filter]) AND (the list of pmids)`
you then export the output as a csv and name it "pmcoalist.csv"

In [98]:
pmcoalist=pd.read_csv('./input/pmcoalist.csv')

In [99]:
pmcoalist=pd.read_csv('./input/pmcoalist.csv')
df=pd.merge(extractioneligible,pmcoalist)
df.to_csv('./output/pmcoalist.csv')

### This new pmcoalist is the input to the Rscript "autotransparent.R"

## Merge codesharing and resttransparency dataframes

In [100]:
codesharing=pd.read_csv('./output/codesharing.csv')
resttransparency=pd.read_csv('./output/resttransparency.csv')
codetransp = codesharing.rename(columns={'pmid':'PMID'})
resttransp = resttransparency.rename(columns={'pmid':'PMID'})

In [101]:
transp=pd.merge(resttransp, codetransp, on='PMID')
transp=pd.merge(df, transp, on='PMID')

In [102]:
all=list(pd.read_csv('~/Downloads/2019_results.csv').PMID)
twentyone=list(pd.read_csv('~/Downloads/2021_results.csv').PMID)
nineteen=list(pd.read_csv('~/Downloads/2019_results.csv').PMID)

all.extend(twentyone)

# Some minor changes to the dataframe

In [103]:
transp=transp[transp['PMID'].isin(list(extractioneligible.PMID))]
transp['Publication Year'][transp['PMID'].isin(nineteen)]='2019'
transp['Publication Year'][transp['PMID'].isin(twentyone)]='2021'

/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1416903862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transp['Publication Year'][transp['PMID'].isin(nineteen)]='2019'
/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1416903862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transp['Publication Year'][transp['PMID'].isin(twentyone)]='2021'


In [104]:
transp['Publication_Year']=transp['Publication Year']
transp['Journal']=transp['Journal/Book']
columns=['PMID','is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Publication_Year','Journal']
transp=pd.DataFrame(transp, columns=columns)
transp['is_open_code'] = transp['is_open_code'].astype('bool')
transp['is_open_data'] = transp['is_open_data'].astype('bool')


#transp["Publication_Year"]=transp["Publication_Year"].values.astype(str)
transp.Publication_Year.iloc[(transp['Publication_Year'].isin(['2021']))& (transp['Disease']!='Covid-19')]='2021 Non Covid-19'
transp.Publication_Year.iloc[(transp['Publication_Year']=='2021')& (transp['Disease']=='Covid-19')]='2021 Covid-19'

/Users/emmanuel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## export the cleaned dataframe to a merged dataframe

In [105]:
transp.to_csv('./results/transparency_indicators.csv')

# For main table of study characteristics

For the counts of the year subsets and their percentages take these values and do the math

In [106]:
transp['Publication_Year'].value_counts()


2021 Covid-19        818
2021 Non Covid-19    304
2019                 216
Name: Publication_Year, dtype: int64

## Models

In [107]:
tjugonitton=transp[transp['Publication_Year']=='2019']
tjugotjugoettcovid=transp[transp['Publication_Year']=='2021 Covid-19']
tjugotjugoettnovid=transp[transp['Publication_Year']=='2021 Non Covid-19']

a=dict(transp.groupby('Publication_Year')['Model'].value_counts())
b=dict(tjugonitton.groupby('Publication_Year')['Model'].value_counts()/len(tjugonitton)*100)
c=dict(tjugotjugoettcovid.groupby('Publication_Year')['Model'].value_counts()/len(tjugotjugoettcovid)*100)
d=dict(tjugotjugoettnovid.groupby('Publication_Year')['Model'].value_counts()/len(tjugotjugoettnovid)*100)


a=pd.DataFrame.from_dict(a,orient='index')
b=pd.DataFrame.from_dict(b,orient='index')
c=pd.DataFrame.from_dict(c,orient='index')
d=pd.DataFrame.from_dict(d,orient='index')
b=pd.concat([b,c])
b=pd.concat([b,d])


pd.merge(a, b, left_index=True, right_index=True).to_csv('./results/main_table_model.csv')


In [108]:
transp['Model'].value_counts()

Compartmental     511
Time Series       337
Spatiotemporal    279
Agent-based       186
Multiple           25
Name: Model, dtype: int64

In [109]:
transp['Model'].value_counts()/len(transp) *100

Compartmental     38.191330
Time Series       25.186846
Spatiotemporal    20.852018
Agent-based       13.901345
Multiple           1.868460
Name: Model, dtype: float64

## Diseases

In [110]:
tjugonitton=transp[transp['Publication_Year']=='2019']
tjugotjugoettcovid=transp[transp['Publication_Year']=='2021 Covid-19']
tjugotjugoettnovid=transp[transp['Publication_Year']=='2021 Non Covid-19']

a=dict(transp.groupby('Publication_Year')['Disease'].value_counts())
b=dict(tjugonitton.groupby('Publication_Year')['Disease'].value_counts()/len(tjugonitton)*100)
c=dict(tjugotjugoettcovid.groupby('Publication_Year')['Disease'].value_counts()/len(tjugotjugoettcovid)*100)
d=dict(tjugotjugoettnovid.groupby('Publication_Year')['Disease'].value_counts()/len(tjugotjugoettnovid)*100)


a=pd.DataFrame.from_dict(a,orient='index')


b=pd.DataFrame.from_dict(b,orient='index')
c=pd.DataFrame.from_dict(c,orient='index')
d=pd.DataFrame.from_dict(d,orient='index')
b=pd.concat([b,c])

b=pd.concat([b,d])


pd.merge(a, b, left_index=True, right_index=True).to_csv('./results/main_table_disease.csv')


## Journals

In [111]:
a=dict(transp.groupby('Publication_Year')['Journal'].value_counts())

tjugonitton=transp[transp['Publication_Year']=='2019']
tjugotjugoettcovid=transp[transp['Publication_Year']=='2021 Covid-19']
tjugotjugoettnovid=transp[transp['Publication_Year']=='2021 Non Covid-19']

b=dict(tjugonitton.groupby('Publication_Year')['Journal'].value_counts()/len(tjugonitton)*100)
c=dict(tjugotjugoettcovid.groupby('Publication_Year')['Journal'].value_counts()/len(tjugotjugoettcovid)*100)
d=dict(tjugotjugoettnovid.groupby('Publication_Year')['Journal'].value_counts()/len(tjugotjugoettnovid)*100)


a=pd.DataFrame.from_dict(a,orient='index')
b=pd.DataFrame.from_dict(b,orient='index')
c=pd.DataFrame.from_dict(c,orient='index')
d=pd.DataFrame.from_dict(d,orient='index')
b=pd.concat([b,c])
b=pd.concat([b,d])


Journals=pd.merge(a, b, left_index=True, right_index=True)
Journals.to_csv('./results/main_table_journal.csv')


In [112]:
dict(transp['Journal'].value_counts())

{'PLoS One': 115,
 'Sci Rep': 91,
 'Int J Environ Res Public Health': 63,
 'BMC Infect Dis': 38,
 'PLoS Negl Trop Dis': 33,
 'PLoS Comput Biol': 29,
 'BMC Public Health': 28,
 'Chaos Solitons Fractals': 25,
 'Infect Dis Model': 23,
 'Results Phys': 22,
 'Front Public Health': 17,
 'medRxiv': 17,
 'Epidemiol Infect': 16,
 'Epidemics': 16,
 'BMJ Open': 13,
 'Nonlinear Dyn': 13,
 'Parasit Vectors': 11,
 'J R Soc Interface': 11,
 'Math Biosci': 10,
 'Vaccines (Basel)': 10,
 'PeerJ': 10,
 'J Math Biol': 10,
 'Nat Commun': 9,
 'Physica A': 9,
 'Proc Natl Acad Sci U S A': 9,
 'Comput Biol Med': 9,
 'Int J Infect Dis': 9,
 'Adv Differ Equ': 9,
 'Vaccine': 9,
 'R Soc Open Sci': 8,
 'Viruses': 8,
 'J Med Internet Res': 8,
 'JAMA Netw Open': 7,
 'Appl Soft Comput': 7,
 'Appl Intell (Dordr)': 7,
 'Eur Phys J Plus': 7,
 'J Theor Biol': 7,
 'JMIR Public Health Surveill': 7,
 'Spat Stat': 6,
 'Healthcare (Basel)': 6,
 'Malar J': 6,
 'Infect Drug Resist': 6,
 'Biology (Basel)': 6,
 'Sustain Cities Soc

In [113]:
dict(round(transp['Journal'].value_counts()/len(transp) *100,1))

{'PLoS One': 8.6,
 'Sci Rep': 6.8,
 'Int J Environ Res Public Health': 4.7,
 'BMC Infect Dis': 2.8,
 'PLoS Negl Trop Dis': 2.5,
 'PLoS Comput Biol': 2.2,
 'BMC Public Health': 2.1,
 'Chaos Solitons Fractals': 1.9,
 'Infect Dis Model': 1.7,
 'Results Phys': 1.6,
 'Front Public Health': 1.3,
 'medRxiv': 1.3,
 'Epidemiol Infect': 1.2,
 'Epidemics': 1.2,
 'BMJ Open': 1.0,
 'Nonlinear Dyn': 1.0,
 'Parasit Vectors': 0.8,
 'J R Soc Interface': 0.8,
 'Math Biosci': 0.7,
 'Vaccines (Basel)': 0.7,
 'PeerJ': 0.7,
 'J Math Biol': 0.7,
 'Nat Commun': 0.7,
 'Physica A': 0.7,
 'Proc Natl Acad Sci U S A': 0.7,
 'Comput Biol Med': 0.7,
 'Int J Infect Dis': 0.7,
 'Adv Differ Equ': 0.7,
 'Vaccine': 0.7,
 'R Soc Open Sci': 0.6,
 'Viruses': 0.6,
 'J Med Internet Res': 0.6,
 'JAMA Netw Open': 0.5,
 'Appl Soft Comput': 0.5,
 'Appl Intell (Dordr)': 0.5,
 'Eur Phys J Plus': 0.5,
 'J Theor Biol': 0.5,
 'JMIR Public Health Surveill': 0.5,
 'Spat Stat': 0.4,
 'Healthcare (Basel)': 0.4,
 'Malar J': 0.4,
 'Infect D

## Get the study characteristics
These include diseases, model types and journal.

In [114]:
diseasepercent=df['Disease'].value_counts().rename_axis('Disease').reset_index(name='Percentage')
diseasepercent['Percentage']=diseasepercent.Percentage.apply(lambda x: round((x/len(df))*100,1))
diseasepercent.to_csv('./results/diseasespercentage.csv',index=False)

models=df['Model'].value_counts().rename_axis('Models').reset_index(name='Percentage')
models['Percentage']=models.Percentage.apply(lambda x: round((x/len(df))*100,1))
models.to_csv('./results/modelpercentage.csv',index=False)

# Get the occurences of the transparency indicators across subgroups of interest

In [115]:
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
model_transp=pd.DataFrame(transpar.groupby('Model').sum())

In [116]:
# Model type
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
model_transp=pd.DataFrame(transpar.groupby('Model').sum())
model_transp.to_csv('./results/model_transparency_occurrences.csv')

# Disease
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'
disease_top_5_transp=pd.DataFrame(transpar.groupby('Disease').sum())
disease_top_5_transp.to_csv('./results/disease_transp_occurrences.csv')

# Year & COVID status
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Publication_Year']
transpar=pd.DataFrame(transp, columns=columns)
Year=pd.DataFrame(transpar.groupby('Publication_Year').sum())
Year.to_csv('./results/year_covid_occurrences.csv')

# Journals= top 3
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Journal']
transpar=pd.DataFrame(transp, columns=columns)
transpar['Journal'][~transpar['Journal'].isin(['PLoS One','Sci Rep','Int J Environ Res Public Health'])]='Other'
journal_transp=pd.DataFrame(transpar.groupby('Journal').sum())
journal_transp.to_csv('./results/journal_transp_occurrences.csv')


/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1442954762.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'
/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1442954762.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transpar['Journal'][~transpar['Journal'].isin(['PLoS One','Sci Rep','Int J Environ Res Public Health'])]='Other'


In [117]:
# Model type
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
model_transp=pd.DataFrame(round(transpar.groupby('Model').mean()*100,1))
model_transp.to_csv('./results/model_transparency_percentages.csv')

# Disease
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'
disease_top_5_transp=pd.DataFrame(round(transpar.groupby('Disease').mean()*100,1))
disease_top_5_transp.to_csv('./results/disease_transp_percentages.csv')

# Year & COVID status
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Publication_Year']
transpar=pd.DataFrame(transp, columns=columns)
Year=pd.DataFrame(round(transpar.groupby(['Publication_Year']).mean()*100,1))
Year.to_csv('./results/year_covid_percentages.csv')

# Journals= top 3
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Journal']
transpar=pd.DataFrame(transp, columns=columns)
transpar['Journal'][~transpar['Journal'].isin(['PLoS One','Sci Rep','Int J Environ Res Public Health'])]='Other'
journal_transp=pd.DataFrame(round(transpar.groupby('Journal').mean()*100,1))
journal_transp.to_csv('./results/journal_transp_percentage.csv')

/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1135825946.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'
/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/1135825946.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transpar['Journal'][~transpar['Journal'].isin(['PLoS One','Sci Rep','Int J Environ Res Public Health'])]='Other'


# Get the percentages of the transparency indicators across subgroups of interest

In [118]:
# Model type
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpas=pd.DataFrame(transp, columns=columns)
model_transp=pd.DataFrame(round(transpas.groupby('Model').mean()*100,1))
model_transp.to_csv('./results/model_transparency.csv')

# Disease
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model']
transpar=pd.DataFrame(transp, columns=columns)
transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'
disease_top_5_transp=pd.DataFrame(round(transpar.groupby('Disease').mean()*100,1))
disease_top_5_transp.to_csv('./results/disease_top_5_transp.csv')

# Year & COVID status
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Publication_Year']
transpar=pd.DataFrame(transp, columns=columns)

Year=pd.DataFrame(round(transpar.groupby(['Publication_Year']).mean()*100,1))
Year.to_csv('./results/year_covid.csv')

# Journals= top 3
columns=['is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Disease','Model','Journal']
transpar=pd.DataFrame(transp, columns=columns)
transpar=transpar[transpar['Journal'].isin(['PLoS One','Sci Rep','Int J Environ Res Public Health'])]
journal_transp=pd.DataFrame(round(transpar.groupby('Journal').mean()*100,1))
journal_transp.to_csv('./results/journal_transp.csv')


/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/3055621466.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transpar['Disease'][~transpar['Disease'].isin(['Covid-19','General (theoretical model)','Malaria','Dengue','Influenza illnesses'])]='Other'


# Transparency indicators across the years for the 2019 and 2021 sample 

In [119]:
tjugotjugoett=transp[transp['Publication_Year'].isin(['2021 Covid-19', '2021 Non Covid-19'])]
tjugotjugoett['Publication_Year'][tjugotjugoett['Publication_Year'].isin(['2021 Covid-19', '2021 Non Covid-19'])]='2021'
tjugotjugoett

# to extract the 2021 overall transparency
(round((tjugotjugoett.groupby('Publication_Year').mean()*100),1)).to_csv('./results/year2021.csv')

/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/3417509419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tjugotjugoett['Publication_Year'][tjugotjugoett['Publication_Year'].isin(['2021 Covid-19', '2021 Non Covid-19'])]='2021'
/Users/emmanuel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


### For all publications

In [120]:
transpas.sum()

is_coi_pred                                                      1197
is_register_pred                                                    6
is_fund_pred                                                     1109
is_open_code                                                      288
is_open_data                                                      332
Disease             Covid-19Covid-19Covid-19Covid-19Covid-19Covid-...
Model               CompartmentalCompartmentalSpatiotemporalTime S...
dtype: object

In [121]:
transpas.mean()*100

/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_32845/735997599.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  transpas.mean()*100


is_coi_pred         89.461883
is_register_pred     0.448430
is_fund_pred        82.884903
is_open_code        21.524664
is_open_data        24.813154
dtype: float64

## Papers sharing neither or both code and data
Extracts all the papers that share code and data, and the ones that share neither

In [122]:
shareboth=transp.loc[(codesharing.is_open_data==True) & (codesharing.is_open_code==True)]
shareboth.to_csv('./output/shareboth.csv')

shareneither=transp.loc[(codesharing.is_open_data==False) & (codesharing.is_open_code==False)]
shareneither.to_csv('./output/shareneither.csv')

In [123]:
shareneitherstr=str(round((len(shareneither)/len(transp))*100,2))
sharebothstr=str(round((len(shareboth)/len(transp))*100,2))

print('The ones that share neither code nor data are '+shareneitherstr+'% of the papers.', 'The ones that share both are '+sharebothstr+'% of the papers')

The ones that share neither code nor data are 68.68% of the papers. The ones that share both are 14.87% of the papers


For the raw number of these studies

In [124]:
len(shareneither), len(shareboth)

(919, 199)

# Validating Code sharing, Data sharing, and Registrations
test 10% of samples that share code 10% that share data and all registrations, EAZ went through the actual statements to validate them.

In [125]:
codesampl=codetransp[codetransp['is_open_code']==True]
codesample=list(codesampl.PMID)
random.seed(1821)
codesearch=random.sample(codesample,int(round(len(codesample)*0.1,0)))
codesampl=codesampl[codesampl['PMID'].isin(codesearch)]
pd.DataFrame(codesampl, columns=['open_code_statements','PMID']).to_csv('./output/validate_code.csv')

In [126]:
datasampl=codetransp[codetransp['is_open_data']==True]
datasample=list(datasampl.PMID)
random.seed(1821)
datasearch=random.sample(datasample,int(round(len(datasample)*0.1,0)))

datasampl=datasampl[datasampl['PMID'].isin(datasearch)]
pd.DataFrame(datasampl, columns=['open_data_statements','PMID']).to_csv('./output/validate_data.csv')

In [127]:
registrationsampl=resttransp[resttransp['is_register_pred']==True]
pd.DataFrame(registrationsampl, columns=['register_text','PMID']).to_csv('./output/validate_register.csv')

## Results

In [128]:
linktesting=pd.read_csv('./output/manual_validation_cleaned_no_COI_fund.csv')

In [129]:
registrationtest=linktesting[linktesting['variable_tested']=='Registration']
codetest=linktesting[linktesting['variable_tested']=='Code']
datatest=linktesting[linktesting['variable_tested']=='Data']

In [130]:
round(codetest.actually_transparent.value_counts()/len(codetest)*100,2)

True     82.76
False    17.24
Name: actually_transparent, dtype: float64

In [131]:
round(datatest.actually_transparent.value_counts()/len(datatest)*100,2)

True     87.88
False    12.12
Name: actually_transparent, dtype: float64

In [132]:
round(registrationtest.actually_transparent.value_counts()/len(registrationtest)*100,2)

True     83.33
False    16.67
Name: actually_transparent, dtype: float64

## characteristics of the registered studies

In [133]:
registrationtestPMID=list(registrationtest['PMID'][registrationtest.actually_transparent])
transp[transp['PMID'].isin(registrationtestPMID)]

,PMID,is_coi_pred,is_register_pred,is_fund_pred,is_open_code,is_open_data,Disease,Model,Publication_Year,Journal
41,34932560,True,True,True,False,True,Polio,Agent-based,2021 Non Covid-19,PLoS Comput Biol
251,34625589,True,True,True,False,False,Malaria,Spatiotemporal,2021 Non Covid-19,Sci Rep
512,34187546,True,True,True,False,False,Vector,Spatiotemporal,2021 Non Covid-19,Parasit Vectors
1173,31568489,True,True,True,False,True,Malaria,Spatiotemporal,2019,PLoS One
1178,31698706,True,True,True,False,False,Rotavirus,Time Series,2019,Int J Environ Res Public Health


# Validating conflict of interest and funding disclosures

Same approach as in the the other validation

In [134]:
COIcolumns= ['PMID','is_coi_pred','coi_text']

In [135]:
COI=resttransp[resttransp['is_coi_pred']==True]
COIsearch=list(COI.PMID)
random.seed(1821)
COIsearch=random.sample(COIsearch,int(round(len(COIsearch)*0.1,0)))
COI=COI[COI['PMID'].isin(COIsearch)]

pd.DataFrame(COI, columns=COIcolumns).to_csv('./output/validation_coi.csv')

### 4 actually had a real conflict

In [136]:
actual_conflict=[33446891, 33878351, 34511711, 34045002, 33886486]

In [137]:
FUNDcolumns=['PMID','fund_text']

In [138]:
FUND=resttransp[resttransp['is_fund_pred']==True]
FUNDsearch=list(FUND.PMID)
random.seed(1821)
FUNDsearch=random.sample(FUNDsearch,int(round(len(FUNDsearch)*0.1,0)))
FUND=FUND[FUND['PMID'].isin(FUNDsearch)]

pd.DataFrame(FUND, columns=FUNDcolumns).to_csv('./output/validation_fund.csv')

### funding statements that had NO funding statement (as in no funding to disclose)

In [139]:
nofund=[31145744, 31209084,33425640,33933228,33455588,33665042,34205809,34483725,34452040,34525112,34658535,34690384,34886507]

### the percentage and no. of studies that disclose nothing in the COI statement

In [140]:
print(str(len(COI)-len(actual_conflict))+' (' +str((1-round(len(actual_conflict)/len(COI),3) )*100)+')')


115 (95.8)


### the percentage and no of studies that disclose nothing in the Funding statement

In [141]:
nofundno=(round(len(nofund)/len(FUND),3))
print(str(len(nofund))+' ('+str(round(nofundno*100,1))+')')

13 (11.7)
